In [3]:
import pandas as pd
import os

In [5]:
# Files to load
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

In [6]:
school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

In [23]:
# Initial exploration
print("Students dataframe dims:\n",student_data_df.count())
print("\nNulls in student dataframe:\n", student_data_df.isnull().sum())
print("\nData types in student dataframe:\n", student_data_df.dtypes)

print("\nSchools dataframe dims:\n",school_data_df.count())
print("\nNulls in schools dataframe:\n", school_data_df.isnull().sum())
print("Data types in schools dataframe:\n", school_data_df.dtypes)


Students dataframe dims:
 Student ID       39170
student_name     39170
gender           39170
grade            39170
school_name      39170
reading_score    39170
math_score       39170
dtype: int64

Nulls in student dataframe:
 Student ID       0
student_name     0
gender           0
grade            0
school_name      0
reading_score    0
math_score       0
dtype: int64

Data types in student dataframe:
 Student ID        int64
student_name     object
gender           object
grade            object
school_name      object
reading_score     int64
math_score        int64
dtype: object

Schools dataframe dims:
 School ID      15
school_name    15
type           15
size           15
budget         15
dtype: int64

Nulls in schools dataframe:
 School ID      0
school_name    0
type           0
size           0
budget         0
dtype: int64
Data types in schools dataframe:
 School ID       int64
school_name    object
type           object
size            int64
budget          int64
dtype:

In [28]:
# Add each prefix and suffix to remove to a list.
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]
# Iterate through the words in the "prefixes_suffixes" list and replace them with an empty space, "".
for word in prefixes_suffixes:
    student_data_df["student_name"] = student_data_df["student_name"].str.replace(word,"")

In [30]:
# Combine the data into a single dataset.
school_data_complete_df = pd.merge(student_data_df, school_data_df, on=["school_name", "school_name"])
school_data_complete_df["school_name"].count()

39170

In [46]:
# Check that the number of unique schools in the merged data is 15
school_count = len(school_data_complete_df["school_name"].unique())
school_count

15

In [41]:
# Get total budget from school dataframe to reference later
total_budget = school_data_df["budget"].sum()
print("All schools total budget: $", total_budget)

All schools total budget: $ 24649428


In [40]:
# Calculate average scores
average_reading_score = school_data_complete_df["reading_score"].mean()
average_math_score = school_data_complete_df["math_score"].mean()
print("All schools average reading score: ", average_reading_score)
print("All schools average math score: ", average_math_score)

All schools average reading score:  81.87784018381414
All schools average math score:  78.98537145774827


In [45]:
# Get all the students who are passing math and reading into new dataframes
passing_math = school_data_complete_df[school_data_complete_df["math_score"] >= 70]
passing_reading = school_data_complete_df[school_data_complete_df["reading_score"] >= 70]
# Count numbers of students passing math, and passing reading
passing_math_count = passing_math["student_name"].count()
passing_reading_count = passing_reading["student_name"].count()
# Count all students
student_count = school_data_complete_df["Student ID"].count()
# Calculate passing percentage
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_reading_percentage = passing_reading_count / float(student_count) * 100
# Create dataframe of students who passed both math and reading
passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70) & (school_data_complete_df["reading_score"] >= 70)]
overall_passing_math_reading_count = passing_math_reading["student_name"].count()
# Calculate the overall passing percentage.
overall_passing_percentage = overall_passing_math_reading_count / student_count * 100

In [53]:
# Create district summary dataframe
district_summary_df = pd.DataFrame(
    [{"Total Schools": school_count,
    "Total Students": student_count,
    "Total Budget": total_budget,
    "Average Math Score": average_math_score,
    "Average Reading Score": average_reading_score,
    "% Passing Math": passing_math_percentage,
    "% Passing Reading": passing_reading_percentage,
    "% Overall Passing": overall_passing_percentage}]
)
# Format the "Total Students" to have the comma for a thousands separator.
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
# Format "Total Budget" to have the comma for a thousands separator, a decimal separator, and a "$".
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
# Format reading score to tenths place
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)
# Format math score to tenths place
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.0f}".format)
# Format % passing reading to whole percentage
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.0f}%".format)
# Format % passing math to nearest whole percentage
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.0f}%".format)
# Format % overall passing to nearest whole percentage
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.0f}%".format)

# Reorder the columns in the order you want them to appear.
new_column_order = ["Total Schools", "Total Students", "Total Budget","Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]

# Assign district summary df the new column order.
district_summary_df = district_summary_df[new_column_order]

# Display dataframe
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",79,81.9,75%,86%,65%


In [65]:
# Per School Analyses
# Get school types
per_school_types = school_data_df.set_index(["school_name"])["type"]
# Add the per_school_types into a DataFrame for testing.
schools_df = pd.DataFrame(per_school_types)

# Find per school student counts from school data df.
# Could also use: school_data_complete_df["school_name"].value_counts()
per_school_counts = school_data_df.set_index(["school_name"])["size"]
per_school_counts

# Calculate the budget for each school
per_school_budget = school_data_df.set_index(["school_name"])["budget"]
per_school_budget

# Calculate the per capita spending.
per_school_capita = per_school_budget / per_school_counts
per_school_capita

# Calculate the average math and reading scores for each school
per_school_math = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]
per_school_reading = school_data_complete_df.groupby(["school_name"]).mean()["reading_score"]
per_school_math

# Calculate the passing scores by creating a filtered DataFrame.
# First create a df of only students who passed math from all schools
per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]
# Then count how many students are in the new df, grouping by school name
per_school_passing_math = per_school_passing_math.groupby(["school_name"]).count()["student_name"]
# Lastly divide the number of students passing math by the total number of students (per school) and * 100 to get the percentage 
per_school_passing_math = per_school_passing_math / per_school_counts * 100

# Repeat to get per school reading percentage
per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]
per_school_passing_reading = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]
per_school_passing_reading = per_school_passing_reading / per_school_counts * 100

# Repeat to get per school overall passing percentage.
per_passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70) & (school_data_complete_df["reading_score"] >= 70)]
per_passing_math_reading = per_passing_math_reading.groupby(["school_name"]).count()["student_name"]
per_overall_passing_percentage = per_passing_math_reading / per_school_counts * 100

school_name
Bailey High School       54.642283
Cabrera High School      91.334769
Figueroa High School     53.204476
Ford High School         54.289887
Griffin High School      90.599455
Hernandez High School    53.527508
Holden High School       89.227166
Huang High School        53.513884
Johnson High School      53.539172
Pena High School         90.540541
Rodriguez High School    52.988247
Shelton High School      89.892107
Thomas High School       90.948012
Wilson High School       90.582567
Wright High School       90.333333
dtype: float64

In [75]:
# Arrange results in output table
per_school_summary_df = pd.DataFrame({
    "School Type": per_school_types,
    "Total Students": per_school_counts,
    "Total School Budget": per_school_budget,
    "Per Student Budget": per_school_capita,
    "Average Math Score": per_school_math,
    "Average Reading Score": per_school_reading,
    "% Passing Math": per_school_passing_math,
    "% Passing Reading": per_school_passing_reading,
    "% Overall Passing": per_overall_passing_percentage})

# Format total and per student budgets to include $, thousandths separator, and round to the hundredths place.
per_school_summary_df["Total School Budget"] = per_school_summary_df["Total School Budget"].map("${:,.2f}".format)
per_school_summary_df["Per Student Budget"] = per_school_summary_df["Per Student Budget"].map("${:,.2f}".format)

#Format average scores to round to tenths place
per_school_summary_df["Average Math Score"] = per_school_summary_df["Average Math Score"].map("{:.1f}".format)
per_school_summary_df["Average Reading Score"] = per_school_summary_df["Average Reading Score"].map("{:.1f}".format)

# Format math, reading, overall passing percents to be whole percentages - commented out because we want the precision for next step
# per_school_summary_df["% Passing Math"] = per_school_summary_df["% Passing Math"].map("{:.0f}%".format)
# per_school_summary_df["% Passing Reading"] = per_school_summary_df["% Passing Reading"].map("{:.0f}%".format)
# per_school_summary_df["% Overall Passing"] = per_school_summary_df["% Overall Passing"].map("{:.0f}%".format)

per_school_summary_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.0,81.0,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.1,84.0,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.7,81.2,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.1,80.7,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.4,83.8,93.392371,97.138965,90.599455


In [81]:
# Find highest performing schools
# Sort and show top five schools.
top_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending=False)

top_schools.head()

# Sort and show top five schools.
bottom_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending=True)

bottom_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.8,80.7,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.7,81.2,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.6,81.2,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.3,80.9,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.1,81.0,66.057551,81.222432,53.539172
